# Intermediate SQL

* Intermediate SQL
	* Multi-table commands: 
        * Join
        * Union
    * Nested table selection syntax
	* Updating data: 
        * insert
        * update 
        * delete
	* Replacing null values
	* Text search: Like, etc
        * Like

![](https://img.pokemondb.net/artwork/ivysaur.jpg)

In [ ]:
import sqlite3 as sq3
import pandas.io.sql as pds
import pandas as pd

path = '../../sql'
con = sq3.Connection(path + 'pokemon.db')

query = '''DROP TABLE IF EXISTS locations;'''
con.execute(query)
query = '''DROP TABLE IF EXISTS pokemon;'''
con.execute(query)
query = '''DROP TABLE IF EXISTS encounters;'''
con.execute(query)
query = '''DROP TABLE IF EXISTS red_pokemon;'''
con.execute(query)
query = '''DROP TABLE IF EXISTS blue_pokemon;'''
con.execute(query)
con.commit()

In [ ]:
# load locations
locarea_df = pd.read_csv('data/location_details.csv', index_col='id')
locarea_df.to_sql('locations', con)

#load pokemon
pokemon_df = pd.read_csv('data/pokemon_details.csv', index_col='id')
pokemon_df.to_sql('pokemon', con)

#load encounters
encounters_df = pd.read_csv('data/encounters.csv', index_col='id')
encounters_df.to_sql('encounters', con)

#load red pokemon
red_df = pd.read_csv('data/red_pokemon.csv', index_col='id')
red_df.to_sql('red_pokemon', con)

#load blue pokemon
blue_df = pd.read_csv('data/blue_pokemon.csv', index_col='id')
blue_df.to_sql('blue_pokemon', con)

In [ ]:
%matplotlib inline
color = 'red'

query = '''
SELECT * 
FROM {0}_pokemon {0}
Join pokemon p on {0}.id = p.id
where {0}.id is not null
'''.format(color)
df = pds.read_sql(query, con)
plt = df.plot('height', 'weight', kind='scatter', alpha=0.5)

## Multi-table commands:

![](https://img.pokemondb.net/artwork/wartortle.jpg)

### Union
Unions allow you to take two queries and concatenate the output for each

In [ ]:
query = '''
SELECT * 
FROM pokemon
where type1 = 'Fire';
'''
pds.read_sql(query, con)

In [ ]:
query = '''
SELECT * 
FROM pokemon
where type2 = 'Flying';
'''
pds.read_sql(query, con).head()

In [ ]:
query = '''
SELECT * 
FROM pokemon
where type1 = 'Fire'
UNION
SELECT * 
FROM pokemon
where type2 = 'Flying';
'''
pds.read_sql(query, con).head()

In [ ]:
#Union All does not delete duplicate rows. So Moltres shows up twice, for example
query = '''
SELECT * 
FROM pokemon
where type1 = 'Fire'
UNION ALL
SELECT * 
FROM pokemon
where type2 = 'Flying';
'''
pds.read_sql(query, con).head()

![](images/SQL_Inner_Joins.png)

In [ ]:

query = '''
SELECT * 
FROM red_pokemon red
JOIN blue_pokemon blue ON red.id=blue.id;
'''
pds.read_sql(query, con).head()

![](images/SQL_Left_Joins.png)

In [ ]:
query = '''
SELECT red.id as red_id,  red.name as red_name,blue.id as blue_id,blue.name as blue_name
FROM red_pokemon red
LEFT JOIN blue_pokemon blue ON red.id=blue.id;'''
pds.read_sql(query, con).head()

![](images/SQL_Right_Joins.png)

In [ ]:
# this would work in most versions of SQL
query = '''
SELECT * 
FROM red_pokemon red
RIGHT JOIN blue_pokemon blue ON red.id=blue.id;'''
#pds.read_sql(query, con)

In [ ]:
# But a Right Join is just a left join with the order reversed
query = '''
SELECT * 
FROM blue_pokemon blue
LEFT JOIN red_pokemon red ON red.id=blue.id;'''
pds.read_sql(query, con).head()

![](images/SQL_Left_Exclusion_Joins.png)

In [ ]:
query = '''
SELECT * 
FROM red_pokemon red
LEFT JOIN blue_pokemon blue ON red.id=blue.id
WHERE blue.id IS NULL'''
pds.read_sql(query, con)

![](images/SQL_Right_Exclusion_Joins.png)

In [ ]:
query = '''
SELECT * 
FROM blue_pokemon blue
LEFT JOIN red_pokemon red ON red.id=blue.id
WHERE red.id IS NULL'''
pds.read_sql(query, con)

![](images/SQL_Outer_Joins.png)

In [ ]:
# This works is many versions of SQL
query = '''
SELECT id,name
FROM red_pokemon red
OUTER JOIN blue_pokemon blue ON red.id=blue.id;
'''
#pds.read_sql(query, con)

# for SQLite we need to do it a different way
query = '''
SELECT null as blue_id , null as blue_name, red.id as red_id,red.name as red_name 
FROM red_pokemon red
UNION
SELECT blue.id as blue_id , blue.name as blue_name, null as red_id,null as red_name  
FROM blue_pokemon blue'''
pds.read_sql(query, con).head(120)

![](images/sqljoins.png)

## Exercises

### Which Pokemon will you not encounter (in the wild) in either Red or Blue?

In [ ]:
%matplotlib inline
query = '''
SELECT p.name , height, weight
FROM pokemon p
LEFT JOIN red_pokemon red ON p.id=red.id
LEFT JOIN blue_pokemon blue ON p.id=blue.id
WHERE red.id is null AND blue.id is null;
'''
missing_pokemon = pds.read_sql(query, con)
missing_pokemon.head()

In [ ]:
missing_pokemon.plot.scatter('height','weight')

### Which Pokemon are encountered in which areas?

In [ ]:
query = '''
SELECT p.name,location_name,area_name, type1
FROM encounters e
JOIN locations l ON l.id=e.location_area_id
JOIN pokemon p on p.id=e.pokemon_id
limit 10;'''
pds.read_sql(query, con).head()

### Which locations have the most electric pokemon?

In [ ]:
query = '''
SELECT location_name,type1, count(*)
FROM encounters e
JOIN locations l ON l.id=e.location_area_id
JOIN pokemon p on p.id=e.pokemon_id
where type1 = 'Electric'
GROUP BY 1,2
ORDER BY 3 DESC'''
pds.read_sql(query, con).head()

### Average Level of each location

In [ ]:
query = '''
SELECT location_name, avg((min_level+max_level)/2) as average_level
FROM encounters e
JOIN locations l ON l.id=e.location_area_id
JOIN pokemon p on p.id=e.pokemon_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 40;'''
pds.read_sql(query, con).head()

### Subqueries
![](https://img.pokemondb.net/artwork/raichu.jpg)

you can treat the output of any SQL Query as if it were a table. you just need to wrap the query in parenthesis

In [ ]:
query = '''
SELECT location_name,type1, count(*)
FROM encounters e
JOIN (SELECT l.id as location_id,location_name, avg((min_level+max_level)/2) as average_level
    FROM encounters e
    JOIN locations l ON l.id=e.location_area_id
    JOIN pokemon p on p.id=e.pokemon_id
    GROUP BY 1,2
    ORDER BY 3 DESC
    LIMIT 40) high_lvl_locations ON high_lvl_locations.location_id=e.location_area_id
JOIN pokemon p on p.id=e.pokemon_id
where type1 = 'Ghost'
GROUP BY 1,2
ORDER BY 3 DESC
'''
pds.read_sql(query, con).head()

### Updating data: 

#### insert
![](https://sqlite.org/images/syntax/insert-stmt.gif)

In [ ]:
query = '''
INSERT INTO pokemon (id, name, height, weight, base_experience, type1, type2) 
VALUES (0,'Missingno', 1, 0.1, 100, null, null);
'''
con.execute(query)
con.commit()

In [ ]:
query = '''
SELECT *
FROM pokemon
WHERE id < 5;'''
pds.read_sql(query, con)

#### update
![](https://sqlite.org/images/syntax/update-stmt.gif)

In [ ]:
query = '''
UPDATE pokemon 
SET type1 = 'Bug'
WHERE id = 0;
'''
con.execute(query)
con.commit()

In [ ]:
query = '''
SELECT *
FROM pokemon
WHERE id < 5;'''
pds.read_sql(query, con)

Make sure and remember the WHERE statement!

In [ ]:
query = '''
UPDATE pokemon 
SET type1 = 'Bug';
'''
con.execute(query)
con.commit()

In [ ]:
query = '''
SELECT *
FROM pokemon
WHERE id < 5;'''
pds.read_sql(query, con)

In [ ]:
query = '''DROP TABLE IF EXISTS pokemon;'''
con.execute(query)
con.commit()
pd.read_csv('data/pokemon_details.csv', index_col='id').to_sql('pokemon', con)

#### create table as
![](https://sqlite.org/images/syntax/create-table-stmt.gif)

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS location_level AS
SELECT l.id,location_name, avg((min_level+max_level)/2) as average_level
FROM encounters e
JOIN locations l ON l.id=e.location_area_id
JOIN pokemon p on p.id=e.pokemon_id
GROUP BY 1'''
con.execute(query)
con.commit()

In [ ]:
query = '''
SELECT *
FROM location_level
LIMIT 10;'''
pds.read_sql(query, con)

### Dealing with null values
![](https://img.pokemondb.net/artwork/gyarados.jpg)

#### Coalesce

In [ ]:
query = '''
SELECT name,type1,coalesce(type2,type1)
FROM pokemon
LIMIT 10;'''
pds.read_sql(query, con)

### Text search
![](https://img.pokemondb.net/artwork/eevee.jpg)

#### Like

In [ ]:
query = '''
SELECT *
FROM pokemon
where name like '%s';'''
pds.read_sql(query, con)